# Model Save & Restore

- 어플리케이션을 개발하는 것과 마찬가지로 텐서플로우로 학습 모델을 만들때에도 운영 환경을 고려하게 된다.
- 즉, 실제 데이터를 가지고 예측 모델을 만드는 과정을 거쳐서 나온 예측 함수를 매번 새로 학습시키는 것이 아니라 만들어진 모델에 맞는 데이터를 저장해서 그냥 쓰게 만드는 것이다.
- 이 예제는 기존에 학습하고 저장한 모델 정보를 불러와서 운영 환경에서 사용할 수 있게 모델을 복원하는 역할을 한다.

In [1]:
# 보통 프로그래밍 언어에서 파일을 다룰때는 사용이 완료된 시점에 반드시 닫아야하는데
# 파이선에서는 이 과정을 with 구문을 이용해서 단순화 시킨다.

with open('workfile', 'wb') as f:
    f.write('hello world')

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
input_data = [[1,5,3,7,8,10,12],
              [5,8,10,3,9,7,1]]
label_data = [[0,0,0,1,0],
              [1,0,0,0,0]]

In [4]:
INPUT_SIZE = 7
HIDDEN1_LAYER_SIZE = 10
HIDDEN2_LAYER_SIZE = 8
CLASSES = 5
LEARNING_RATE = 0.5

- 데이터를 텐서로 변환하는 작업을 진행한다.
- 텐서에서 shape 의 첫번째는 배치사이즈로 보통 정의한다.
- 보통 배치 사이즈를 사전에 알기는 어렵기 때문에 None 으로 정의하기도 한다.

In [5]:
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE], name = 'x')
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES], name = 'y_')

In [6]:
tensor_map  = {x: input_data, y_:label_data}

In [7]:
 wh1 = tf.Variable( tf.truncated_normal( shape=[INPUT_SIZE, HIDDEN1_LAYER_SIZE], dtype=tf.float32) , name = 'wh1')
bh1 = tf.Variable( tf.zeros([HIDDEN1_LAYER_SIZE]), dtype=tf.float32, name = 'bh1')

wh2 = tf.Variable( tf.truncated_normal( shape=[HIDDEN1_LAYER_SIZE, HIDDEN2_LAYER_SIZE], dtype=tf.float32) , name = 'wh2') #
bh2 = tf.Variable( tf.zeros([HIDDEN2_LAYER_SIZE]), dtype=tf.float32, name = 'bh2')

wo1 = tf.Variable( tf.truncated_normal( shape=[HIDDEN2_LAYER_SIZE, CLASSES], dtype=tf.float32) , name = 'wo1')
bo1 = tf.Variable( tf.zeros([CLASSES]), dtype=tf.float32, name = 'bo1')

In [8]:
parameter_list = [wh1, bh1, wh2, bh2, wo1, bo1]
saver = tf.train.Saver(parameter_list)

In [9]:
with tf.name_scope('hidden_layer_1') as hidscope1:
    hidden1 = tf.sigmoid(tf.matmul(x, wh1) + bh1, name = 'hidden1')

with tf.name_scope('hidden_layer_2') as hidscope2:
    hidden2 = tf.sigmoid(tf.matmul(hidden1, wh2) + bh2, name = 'hidden2')

with tf.name_scope('output_layer') as outscope:
    y = tf.sigmoid(tf.matmul(hidden2, wo1) + bo1, name = 'y')

기존의 방식대로 초기값 상태로 예측을 진행하면 정상적인 결과를 얻어오지 못한다.

In [10]:
with tf.Session() as sess:
    saver.restore(sess, './tensorflow.ckpt')
    result = sess.run( y, tensor_map )
    print result
    
    # sess.close()

[[  2.80734821e-04   8.33140657e-05   1.20340097e-04   9.99722302e-01
    9.03778055e-05]
 [  9.99789655e-01   7.43806813e-05   3.86880638e-05   1.99923292e-04
    6.25486427e-05]]


In [11]:
# sess.run( tf.global_variables_initializer() )
# result = sess.run( y, tensor_map )
# print result


- https://www.tensorflow.org/api_docs/python/tf/train/Saver
- 체크포인트 값을 로드하는 과정중에 에러가 발생한다면 대부분의 경우는 텐서의 이름이 저장할때와 다르기 때문에 발생하는 경우가 많다.
- 가중치가 없더라도 모델은 동작하지만 우리가 원하는 결과를 얻지 못하는게 문제이다.

# Do training more using trained weight

In [12]:
with tf.name_scope('cost_calculation') as cost_calc:
    cost = tf.reduce_sum(-y_ * tf.log(y) - (1-y_) * tf.log(1-y), axis=1)
    cost = tf.reduce_mean( cost )

with tf.name_scope('training') as training:
    train = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost)

with tf.name_scope('accuracy_calcuation') as acc_calc:
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


- TensorBoard를 사용할때는 namescope를 지정하고 모델 관련된 내용을 학습시켜야 데이터를 visualization 할 수 있다.
  https://www.tensorflow.org/get_started/summaries_and_tensorboard

In [13]:
# tf.Session() 함수도 결국 OS 에 있는 파일을 여는 것과 동일하기 때문에 with 구문을 사용할 수 있다.
# unix, linux 와 같은 posix 운영체제에서는 디바이스를 파일로 간주하고 동작한다는 것을 생각하면 된다.

with tf.Session() as sess:
    saver.restore(sess, './tensorflow.ckpt')

    # Namespace 별로 저장된 정보를 병합한다.
    merged = tf.summary.merge_all()

    for i in range(1000):
        _, loss, acc = sess.run([train, cost, accuracy], tensor_map)

        if not i % 100:
            # 병합된 모델 학습정보를 파일로 쓴다.
            train_writer = tf.summary.FileWriter('./summaries/', sess.graph)

            # 텐서플로우 변수를 저장하는 방법이다. 문제는 이때 변수가 저장되는 기준이
            # 각 변수의 선언에서 변수로 받는 name 을 기준으로 넣는다. 따라서, 반드시
            # 변수를 선언할때는 이름을 선언을 해주는 것이 좋다.
            saver.save(sess, './tensorflow.ckpt')

            print "--------------------"
            print "step :", i
            print "loss :", loss
            print "accu :", acc

    # 연산이 종료되면 세션은 반드시 종료한다.
    # 하지만 with 구문으로 열었을 경우에는 close를 신경쓰지 않아도 된다.
    # sess.close()


--------------------
step : 900
loss : 0.000637258
accu : 1.0


--------------------
step : 800
loss : 0.000645396
accu : 1.0


--------------------
step : 700
loss : 0.000653802
accu : 1.0


--------------------
step : 600
loss : 0.000662416
accu : 1.0


--------------------
step : 500
loss : 0.000671269
accu : 1.0


--------------------
step : 400
loss : 0.00068042
accu : 1.0


--------------------
step : 300
loss : 0.00068966
accu : 1.0


--------------------
step : 200
loss : 0.000699259
accu : 1.0


--------------------
step : 100
loss : 0.000709155
accu : 1.0


--------------------
step : 0
loss : 0.00071923
accu : 1.0


# Launching TensorBoard

To run TensorBoard, use the following command (alternatively python -m tensorflow.tensorboard)
'''
tensorboard --logdir=path/to/log-directory
'''


'''
tensorboard --logdir=./
WARNING:tensorflow:Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
WARNING:tensorflow:Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
WARNING:tensorflow:Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
WARNING:tensorflow:Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
WARNING:tensorflow:Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Starting TensorBoard 41 on port 6006
(You can navigate to http://100.114.52.229:6006)
'''

![TensorBoard1](./images/TensorBoard1.png)
![TensorBoard2](./images/TensorBoard2.png)